In [2]:
#### INPUT PARAMETERS:
qmassratio =  1.75  # m2/m1; by convention must be >= 1
nr         =  10.8  # Orbital separation
# Dimensionless spin parameters of each black hole, assuming
#   the black holes are initially orbiting on the xy-plane
#   and situated on the x-axis, with the center-of-mass at
#   x=y=z=0. Black hole 1 is at x>0; and black hole 2 at x<0.
nchi1x     = +0.   # chi_{1x}, x-component of dimensionless spin for black hole 1 (instantaneously at x>0)
nchi1y     = +0.   # chi_{1y}, y-component of dimensionless spin for black hole 1 (instantaneously at x>0)
nchi1z     = +0.6  # chi_{1z}, z-component of dimensionless spin for black hole 1 (instantaneously at x>0)
nchi2x     = +0.   # chi_{2x}, x-component of dimensionless spin for black hole 2 (instantaneously at x<0)
nchi2y     = +0.   # chi_{2y}, y-component of dimensionless spin for black hole 2 (instantaneously at x<0)
nchi2z     = +0.6  # chi_{2z}, z-component of dimensionless spin for black hole 2 (instantaneously at x<0)

##########################################################
#### DON'T TOUCH; see output after running this cell. ####
from NRPyPN_shortcuts import eval__P_t__and__P_r

# Compute nPt, the tangential component of orbital momentum vector (output = numerical), as well as
#         nPr, the radial component of orbital momentum vector (output = numerical)
nPt, nPr = eval__P_t__and__P_r(qmassratio, nr,
                               nchi1x, nchi1y, nchi1z,
                               nchi2x, nchi2y, nchi2z)

print("If the black holes are situated on the x-axis, and orbiting instantaneously in the")
print("   xy-plane, then one could have P_y=+|P_t| and P_x=-|P_r| for the x>0 black hole,")
print("   and P_y=-|P_t| and P_x=+|P_r| for the x<0 black hole.  Other configurations are")
print("   possible provided one permutes the coordinates consistently with a right-handed")
print("   coordinate system.")
print("|P_t| = ",nPt)
print("|P_r| = ",nPr)

ModuleNotFoundError: No module named 'NRPyPN_shortcuts'

In [1]:
import os
import tempfile
import warnings
from pathlib import Path

import numpy as np
import pytest
from numpy.testing import assert_allclose

# from einsteinpy import geodesic as epyg
import einsteinpy_geodesics as epyg


@pytest.fixture
def input_timelike():
    # Equatorial Spiral Capture
    q = [4.0, np.pi / 2, 0.0]
    p = [0.0, 0.0, 3.5]
    params = [0.0, 0.9476286192385706, -1.0]
    end_lambda = 1.0
    step_size = 0.05

    return q, p, params, end_lambda, step_size


@pytest.fixture
def input_nulllike():
    # Reverse & Capture
    q = [2.5, np.pi / 2, 0.0]
    p = [0.0, 0.0, -2.0]
    params = [0.9, 0.1878809905821554, 0.0]
    end_lambda = 10.0
    step_size = 0.05

    return q, p, params, end_lambda, step_size


def test_csvs_exist_after_julia_call(input_timelike):
    q, p, params, end_lambda, step_size = input_timelike

    lambdas, vecs = epyg.solveSystem(q, p, params, end_lambda, step_size)

    out_path = Path(tempfile.gettempdir()) / "epy_geod_jl_temp"

    list_of_files = os.listdir(out_path)

    assert "lambdas.csv" in list_of_files
    assert "vecs.csv" in list_of_files


def test_return_types(input_timelike):
    q, p, params, end_lambda, step_size = input_timelike

    lambdas, vecs = epyg.solveSystem(q, p, params, end_lambda, step_size)

    assert type(lambdas) == np.ndarray
    assert type(vecs) == np.ndarray
    assert type(lambdas[-1]) in (float, np.float64)
    assert type(vecs[-1, -1]) in (float, np.float64)


def test_geodesic_attributes(input_nulllike):
    q, p, params, end_lambda, step_size = input_nulllike

    lambdas, vecs = epyg.solveSystem(q, p, params, end_lambda, step_size)

    assert lambdas.shape[0] == vecs.shape[0]
    assert vecs.shape[1] == 6


def test_integration_termination_callback(input_nulllike):
    q, p, params, end_lambda, step_size = input_nulllike

    with warnings.catch_warnings(record=True) as w:
        warnings.simplefilter("always")

        lambdas, vecs = epyg.solveSystem(q, p, params, end_lambda, step_size)

        assert len(w) == 1  # 1 warning to be shown
        assert issubclass(w[-1].category, RuntimeWarning)


def test_motion_constant_timelike(input_timelike):
    q, p, params, end_lambda, step_size = input_timelike

    L = p[-1]

    lambdas, vecs = epyg.solveSystem(q, p, params, end_lambda, step_size)

    assert_allclose(L, vecs[:, 5], atol=1e-3, rtol=1e-3)


def test_motion_constants_nulllike(input_nulllike):
    q, p, params, end_lambda, step_size = input_nulllike

    L = p[-1]

    lambdas, vecs = epyg.solveSystem(q, p, params, end_lambda, step_size)

    assert_allclose(L, vecs[:, 5], atol=1e-3, rtol=1e-3)

# A specific case, moved from EinsteinPy's tests
def test_schwarzschild_precession():
    q = [40., np.pi / 2, 0.]
    p = [0., 0., 3.83405]
    params = [0., 0.9791466126276669, -1.]
    end_lambda = 2000.
    step_size = 0.5

    lambdas, vecs = epyg.solveSystem(q, p, params, end_lambda, step_size)

    assert_allclose(vecs[:, 1], np.pi/2, atol=1e-12, rtol=1e-12)
    assert_allclose(vecs[:, 4], 0, atol=1e-12, rtol=1e-12)

In [2]:
# Equatorial Spiral Capture
q = [4.0, np.pi / 2, 0.0]
p = [0.0, 0.0, 3.5]
params = [0.0, 0.9476286192385706, -1.0]    # [a, E, mu] - spin, energy, (rest) mass
end_lambda = 1.0                            # Affine parameter value, wehre integration wild stop
step_size = 0.5


lambdas, vecs = epyg.solveSystem(q, p, params, end_lambda, step_size)
# lambdas - affine parameter values, where integration was performed
# vecs    - 2D Array, containing integrated 3-Positions and Covariant 3-Momenta